In [136]:
import spacy
from spacy.matcher import Matcher 
import json
import pandas as pd
import nltk
import random
import string
from spacy import displacy
!python -m spacy download en_core_web_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-04 20:33:04.806824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 20:33:04.806983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 20:33:04.807010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [137]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [138]:

df = pd.read_csv('dataset.csv',names = ['intents','extents'])
df.head()

,intents,extents
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [139]:
nlp = spacy.load("en_core_web_md")

In [203]:
def get_entities(pipe,text):
  doc = pipe(text)
  entities = {}
  for entity in doc.ents:
    if entity.label_ not in entities:

      entities[entity.label_] = [entity.text]
    else:
      entities[entity.label_] += [entity.text]
  # Show entities in pretty manner
  displacy.render(doc, jupyter=True, style='ent')
  return entities
nlp = spacy.load("en_core_web_md")
text = """i want to fly from boston at 838 am and arrive in denver at 1110 in the morning"""
text = "show the flights from pittsburgh to san francisco again on monday"
ents = get_entities(nlp, text)
print(ents)

{'GPE': ['pittsburgh', 'san francisco'], 'DATE': ['monday']}


In [204]:
parsed_sentence = nlp(text)
lis = [token.text for token in parsed_sentence]
print(lis)
entities = {}
for entity in parsed_sentence.ents:
  if entity.label_ not in entities:
    if entity.label_ == 'GPE':
      loc = entity.text
      loc = loc.split(' ')
      if len(loc)>1:
        id = lis.index(loc[0])
      else:

        id = lis.index(entity.text)
      print(id)
      entities[entity.label_] = [lis[id-1]+' '+entity.text]
  else:
    if entity.label_ == 'GPE':
      loc = entity.text
      loc = loc.split(' ')
      if len(loc)>1:
        id = lis.index(loc[0])
      else:

        id = lis.index(entity.text)
      #id = lis.index(entity.text)
      entities[entity.label_] += [lis[id-1]+' '+entity.text]
entities

['show', 'the', 'flights', 'from', 'pittsburgh', 'to', 'san', 'francisco', 'again', 'on', 'monday']
4


{'GPE': ['from pittsburgh', 'to san francisco']}

In [205]:
def locations(gpe):
  for i in list(entities.keys()):
    if i=='GPE':
      x = entities[i]
      if len(x)>1:
        for j in x:
          loc = j.split(' ')
          if type(entities[i]) is not dict:
            entities[i]={}
            entities[i][loc[0]] = loc[1:]
          else:
            entities[i][loc[0]] = loc[1:]

  return entities

In [206]:
entities = locations(entities)
entities['locations'] = entities.pop('GPE')
entities

{'locations': {'from': ['pittsburgh'], 'to': ['san', 'francisco']}}

In [207]:
ents['locations'] = entities['locations']
ents.pop('GPE')
ents

{'DATE': ['monday'],
 'locations': {'from': ['pittsburgh'], 'to': ['san', 'francisco']}}

###Intents

In [208]:
#Method 1 for finding intents
#Use root + dobj to find intents
def semantic_parsing(request):
  doc = nlp(request)
  intent = ''
  for token in doc:
    if token.dep_ =="dobj":
      intent = token.head.lemma_ + token.lemma_.capitalize()
  #if dobj is not present return 'NoIntentFound'
  if len(intent)>0:
    return intent
  else:
    return 0
intent = semantic_parsing(text)
print(intent)

showFlight


In [209]:
#using traing data for finding verb dobj and nsubj
train = df.sample(frac=0.7, random_state=42)


In [210]:
#removing the atis from intents 
intents = list(train['intents'])
ints= []
for i in intents:
  ints.append(i[5:])
temp = []
# removing duplicates via Naive Method
for element in ints:
  elem = str(element)
  if '#' in elem:
    pass
  elif(elem not in temp):
    temp.append(str(element))
temp

['flight',
 'airfare',
 'airport',
 'aircraft',
 'ground_service',
 'airline',
 'flight_no',
 'distance',
 'abbreviation',
 'capacity',
 'quantity',
 'city',
 'flight_time',
 'restriction',
 'meal',
 'ground_fare',
 'cheapest']

In [211]:
#extracting all the first dobj's that exist in out dataset
def get_dobj(uterence):
  objs = {}
  for i in uterence:
    doc = nlp(i)
    for token in doc:
      if token.dep_ =='dobj':
        if token.text not in objs:
          objs[token.text] = 1
        else:
          objs[token.text] += 1
        break
  dobjs = dict(sorted(objs.items(), key=lambda item: item[1],reverse=True))
  dobj = tuple(dobjs.keys())
  dsubs = []
  for i in dobj:
    if len(i)>3:
      dsubs.append(i)
    else:
      pass
  dsubs = tuple(dsubs)
  return dsubs

In [212]:
#extracting all the first verb that exist in out dataset
def get_tverb(uterence):
  verbs = {}
  for i in uterence:
    doc = nlp(i)
    for token in doc:
      if token.pos_ =='VERB':
        if token.text not in verbs:
          verbs[token.text] = 1
        else:
          verbs[token.text] += 1
        break
  tverbs = dict(sorted(verbs.items(), key=lambda item: item[1],reverse=True))
  nsubj = tuple(tverbs.keys())
  nsubs = []
  for i in nsubj:
    if len(i)>3:
      nsubs.append(i)
    else:
      pass
  tverb = tuple(nsubs)
  return tverb

In [213]:
#extracting all the first nsubj's that exist in out dataset
def get_nsub(uterence):
  sub = {}
  for i in uterence:
    doc = nlp(i)
    for token in doc:
      if token.dep_ =='nsubj':
        if token.text not in sub:
          sub[token.text] = 1
        else:
          sub[token.text] += 1
        break

  nsubs = dict(sorted(sub.items(), key=lambda item: item[1],reverse=True))
  nsubj = tuple(nsubs.keys())
  nsubs = []
  for i in nsubj:
    if len(i)>3:
      nsubs.append(i)
    else:
      pass
  nsubs = tuple(nsubs)
  return nsubs

In [214]:
#storing verb+nsubj+dobj as wordlist
utterences = list(df['extents'])
nsubs,objs,verbs = get_nsub(utterences),get_dobj(utterences),get_tverb(utterences)
nsubs[:10],objs[:10],verbs[:10]

(('flights',
  'what',
  'that',
  'flight',
  'airlines',
  'which',
  'transportation',
  'fare',
  'fares',
  'airline'),
 ('flights',
  'flight',
  'what',
  'information',
  'fares',
  'airlines',
  'fare',
  'list',
  'transportation',
  'which'),
 ('show',
  'like',
  'need',
  'list',
  'give',
  'want',
  'have',
  'leaving',
  'philadelphia',
  'find'))

In [215]:
objs= objs+tuple(temp)

In [216]:
#Method 2
def parse(text,objs,verbs):
  doc = nlp(text)
  dObj =None
  tVerb = None
  # Extract the direct object and its transitive verb
  for token in doc:
    if token.dep_ == "dobj":
      dObj = token
      tVerb = token.head
  # Extract the helper verb
  intentVerb = None
  verbList = verbs
  if tVerb.text in verbList:
    intentVerb = tVerb
  else:
    if tVerb.head.dep_ == "ROOT":
      helperVerb = tVerb.head
  # Extract the object of the intent
  intentObj = None
  objList = objs
  if dObj.text in objList:
    intentObj = dObj
  else:
    for child in dObj.children:
      if child.dep_ == "prep":
        intentObj = list(child.children)[0]
        break
      elif child.dep_ == "compound":
        intentObj = child
        break
  if intentVerb == None or intentObj ==None:
    return 0
  else:

    intent = intentVerb.text + intentObj.text.capitalize()
    return intent
result2 = parse(text,objs, verbs)
print(result2)

showFlights


In [217]:
#Finsing synonyms of all the verb, subj and object and storing as tuples in a list
def get_some_word_synonyms(word):
    word = word.lower()
    synonyms = []
    synsets = wordnet.synsets(word)
    if (len(synsets) == 0):
        return []
    synset = synsets[0]
    lemma_names = synset.lemma_names()
    for lemma_name in lemma_names:
        lemma_name = lemma_name.lower().replace('_', ' ')
        if (lemma_name != word and lemma_name not in synonyms):
            synonyms.append(lemma_name)
    return synonyms
def get_all_word_synonyms(word):
    word = word.lower()
    synonyms = []
    synsets = wordnet.synsets(word)
    if (len(synsets) == 0):
        return []
    for synset in synsets:
        lemma_names = synset.lemma_names()
        for lemma_name in lemma_names:
            lemma_name = lemma_name.lower().replace('_', ' ')
            if (lemma_name != word and lemma_name not in synonyms):
                synonyms.append(lemma_name)
    return synonyms
def make_synonyms(lists):
  syns = []
  ob = []
  for word in lists:
      synonyms = get_all_word_synonyms(word)
      for synonym in synonyms:
          syns.append(synonym)
      ob.append(tuple(syns))
      syns = []
  return ob

In [219]:
nltk.download('wordnet')
from nltk.corpus import sentiwordnet as swn, wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [220]:
ver_syn,obj_syn,sub_sun = make_synonyms(verbs),make_synonyms(objs),make_synonyms(nsubs) 
ver_syn[:2],obj_syn[:2],sub_sun[:2]

([('display',
   'appearance',
   'demo',
   'exhibit',
   'present',
   'demonstrate',
   'prove',
   'establish',
   'shew',
   'testify',
   'bear witness',
   'evidence',
   'picture',
   'depict',
   'render',
   'express',
   'evince',
   'indicate',
   'point',
   'designate',
   'show up',
   'read',
   'register',
   'record',
   'usher'),
  ('the like',
   'the likes of',
   'ilk',
   'wish',
   'care',
   'similar',
   'same',
   'alike',
   'comparable',
   'corresponding')],
 [('flight',
   'flying',
   'flight of stairs',
   'flight of steps',
   'escape',
   'trajectory',
   'fledge'),
  ('flying',
   'flight of stairs',
   'flight of steps',
   'escape',
   'trajectory',
   'fledge')],
 [('flight',
   'flying',
   'flight of stairs',
   'flight of steps',
   'escape',
   'trajectory',
   'fledge'),
  ()])

In [221]:
def semantic_parsing(text,obj_syn,ver_syn):
  doc = nlp(text)
  intents = []
  transVerbs = ver_syn
  directObjs = obj_syn
  for sent in doc.sents:
    tverb = ''
    dobj = ''
    intent = ''
    for token in sent:
      if token.dep_ == 'dobj':
        tverb = token.head.text 
        dobj = token.text
    
    verbSyns = [tpl for tpl in transVerbs if tverb in tpl]
    dobjSyns = [tp for tp in directObjs if dobj in tp]
    if dobjSyns == []:
      intent = verbSyns[0][0].capitalize() +'_' + dobj
    elif verbSyns != [] and dobjSyns != []: 
      intent = verbSyns[0][0].capitalize() +'_' + dobjSyns[0][0].capitalize()
    if intent != '':
      intents.append(tuple((sent.text.rstrip(), intent)))
  return intents
result = semantic_parsing(text,obj_syn,ver_syn)
print(result)

[('show the flights from pittsburgh to san francisco again on monday', 'Show_flights')]


In [222]:
final = {}
final['utterence'] = str(text)
final['intent'] = intent
final['entities'] = ents
final

{'utterence': 'show the flights from pittsburgh to san francisco again on monday',
 'intent': 'showFlight',
 'entities': {'DATE': ['monday'],
  'locations': {'from': ['pittsburgh'], 'to': ['san', 'francisco']}}}

In [223]:
jss = json.dumps(final, ensure_ascii=False, indent=4)
print(jss)

{
    "utterence": "show the flights from pittsburgh to san francisco again on monday",
    "intent": "showFlight",
    "entities": {
        "DATE": [
            "monday"
        ],
        "locations": {
            "from": [
                "pittsburgh"
            ],
            "to": [
                "san",
                "francisco"
            ]
        }
    }
}
